In [1]:
import pandas as pd

# 파일 로드
file = r"C:\Jimin\pharmaLex_sentinel\data\요양심사약제_후처리.xlsx"
df = pd.read_excel(file)

# '�' 포함된 셀 찾기
mask = df.applymap(lambda x: isinstance(x, str) and '�' in x)

# 위치와 값 출력
errors = []
for col in df.columns:
    for idx in df.index[mask[col]]:
        errors.append([idx, col, df.at[idx, col]])

errors_df = pd.DataFrame(errors, columns=["row", "column", "value"])

# 빈도표 (컬럼별 건수)
freq = errors_df['column'].value_counts()

# 저장
errors_df.to_csv("unit_errors_full_list.csv", index=False, encoding="utf-8-sig")
freq.to_csv("unit_errors_summary.csv", encoding="utf-8-sig")

print("총 오류 건수:", len(errors_df))
print(freq)


총 오류 건수: 54
column
세부인정기준 및 방법    45
구분              9
Name: count, dtype: int64


C:\Users\sokch\AppData\Local\Temp\ipykernel_27536\3106701975.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = df.applymap(lambda x: isinstance(x, str) and '�' in x)


In [2]:
# -*- coding: utf-8 -*-
import os
import pandas as pd

excel_path = r"C:\Jimin\pharmaLex_sentinel\data\요양심사약제_후처리.xlsx"
out_dir = os.path.join(os.path.dirname(excel_path), "out_scan")
os.makedirs(out_dir, exist_ok=True)

full_list_csv     = os.path.join(out_dir, "invalid_char_full_list.csv")
by_sheet_csv      = os.path.join(out_dir, "invalid_char_by_sheet.csv")
by_sheet_col_csv  = os.path.join(out_dir, "invalid_char_by_sheet_col.csv")

target_char = "�"  # U+FFFD replacement

def scan_workbook(path):
    xls = pd.ExcelFile(path)
    rows = []

    for sheet in xls.sheet_names:
        # 모든 값을 문자열로 강제 (깨짐 탐지 정확도 ↑)
        df = pd.read_excel(path, sheet_name=sheet, dtype=str)
        for r_idx in range(len(df)):
            for col in df.columns:
                val = df.iat[r_idx, df.columns.get_loc(col)]
                if pd.isna(val):
                    continue
                s = str(val)
                if target_char in s:
                    cnt = s.count(target_char)  # 셀 내 다중 발생 집계
                    rows.append({
                        "sheet": sheet,
                        "row": r_idx + 2,      # 엑셀 표시행(헤더 보정)
                        "column": col,
                        "count_in_cell": cnt,
                        "value": s[:200]       # 너무 길면 미리보기 200자
                    })
    return pd.DataFrame(rows)

df = scan_workbook(excel_path)

# 저장 & 요약
df.to_csv(full_list_csv, index=False, encoding="utf-8-sig")

if df.empty:
    print("검출 0건")
else:
    # 시트별 합계(다중 발생 합계)
    by_sheet = df.groupby("sheet")["count_in_cell"].sum().reset_index().sort_values("count_in_cell", ascending=False)
    by_sheet.to_csv(by_sheet_csv, index=False, encoding="utf-8-sig")

    # 시트-컬럼 피벗(다중 발생 합계)
    by_sheet_col = df.pivot_table(index="sheet", columns="column", values="count_in_cell", aggfunc="sum", fill_value=0)
    by_sheet_col.to_csv(by_sheet_col_csv, encoding="utf-8-sig")

    print("총 발생 개수(셀 내 중복 포함):", int(df["count_in_cell"].sum()))
    print("총 셀 개수(하나 이상 포함한 셀 수):", len(df))
    print("상위 10행 미리보기:")
    print(df.head(10))
    print("\n요약(시트별):")
    print(by_sheet.head(10))


총 발생 개수(셀 내 중복 포함): 94
총 셀 개수(하나 이상 포함한 셀 수): 54
상위 10행 미리보기:
    sheet  row       column  count_in_cell  \
0  Sheet1   34  세부인정기준 및 방법              1   
1  Sheet1   37  세부인정기준 및 방법              4   
2  Sheet1  124  세부인정기준 및 방법              2   
3  Sheet1  131  세부인정기준 및 방법              1   
4  Sheet1  132  세부인정기준 및 방법              1   
5  Sheet1  133  세부인정기준 및 방법              1   
6  Sheet1  141  세부인정기준 및 방법              1   
7  Sheet1  142  세부인정기준 및 방법              1   
8  Sheet1  147  세부인정기준 및 방법              2   
9  Sheet1  153  세부인정기준 및 방법              3   

                                               value  
0  1. 만성신부전증환자(혈액투석 및 복막투석환자) 가. 혈압강하제(고혈압 치료제) 1...  
1  1. 허가사항 범위 내에서 아래와 같은 기준으로 투여시 요양 급여를 인정하며, 동 ...  
2  허가사항 범위 내에서 아래와 같은 기준으로 투여 시 요양 급여를 인정하며, 동 인정...  
3  허가사항 범위 내에서 아래와 같은 기준으로 투여 시 요양 급여를 인정하며, 동 인정...  
4  1. 허가사항 범위 내에서 아래와 같은 기준으로 투여 시 요양급여를 인정하며, 동 ...  
5  1. 허가사항 범위 내에서 아래와 같은 기준으로 투여 시 요양급여를 인정하며, 동 ...  
6  1. 허가사항 범위 내에서 아래와 같은 기준으로 투여 시 요양급여를 인정하며,

In [3]:
import pandas as pd

file = r"C:\Jimin\pharmaLex_sentinel\data\요양심사약제_후처리.xlsx"
df = pd.read_excel(file, dtype=str)

total_chars = 0
cells_with_err = 0

for col in df.columns:
    for val in df[col].dropna():
        if "�" in val:
            cnt = val.count("�")
            total_chars += cnt
            cells_with_err += 1

print("셀 수:", cells_with_err)   # 형님이 본 94
print("문자 수:", total_chars)   # 형님이 찾기에서 본 1655

셀 수: 54
문자 수: 94


In [4]:
import pandas as pd

file = r"C:\Jimin\pharmaLex_sentinel\data\요양심사약제_후처리.xlsx"
df = pd.read_excel(file, dtype=str)

chars = {}

for col in df.columns:
    for val in df[col].dropna():
        for ch in val:
            if ord(ch) > 127:  # 비ASCII 문자만 체크
                chars[ord(ch)] = chars.get(ord(ch), 0) + 1

for code, cnt in sorted(chars.items()):
    print(hex(code), chr(code), cnt)


0xb0 ° 12
0xb2 ² 8
0xb5 µ 6
0xb7 · 455
0xb9 ¹ 2
0xd7 × 4
0x2bc ʼ 1
0x3b1 α 4
0x3bc μ 1
0x2013 – 6
0x2018 ‘ 3
0x2019 ’ 3
0x2020 † 1
0x203b ※ 322
0x2076 ⁶ 4
0x2082 ₂ 1
0x2192 → 9
0x2264 ≤ 23
0x2265 ≥ 93
0x2300 ⌀ 7
0x2460 ① 42
0x2461 ② 37
0x2462 ③ 19
0x2463 ④ 19
0x2464 ⑤ 15
0x2465 ⑥ 6
0x2466 ⑦ 4
0x2467 ⑧ 4
0x2468 ⑨ 10
0x25cb ○ 295
0x3001 、 1
0x3002 。 11
0x300c 「 78
0x300d 」 79
0x300e 『 1
0x304f く 7
0x338d ㎍ 2
0x4e00 一 1
0x5168 全 1
0x5668 器 1
0x571f 土 1
0x75b9 疹 1
0x75d2 痒 1
0x809d 肝 1
0x80c3 胃 1
0x81d3 臓 1
0xac00 가 4693
0xac01 각 223
0xac04 간 950
0xac08 갈 21
0xac10 감 600
0xac11 갑 27
0xac12 값 488
0xac15 강 216
0xac16 갖 4
0xac19 같 609
0xac1c 개 919
0xac1d 객 33
0xac31 갱 7
0xac70 거 494
0xac74 건 324
0xac77 걷 1
0xac78 걸 30
0xac80 검 360
0xac81 겁 2
0xac83 것 73
0xac8c 게 186
0xac90 겐 11
0xac94 겔 4
0xaca8 겨 1
0xaca9 격 125
0xacac 견 195
0xacb0 결 386
0xacb9 겹 1
0xacbd 경 3640
0xacc4 계 319
0xace0 고 1708
0xace1 곡 4
0xace4 곤 54
0xace8 골 300
0xacf0 곰 1
0xacf5 공 176
0xacfc 과 1401
0xacfd 곽 1
0xad00 관 1061
0xad04